Spark Streaming - Twitter

In [ ]:
#Pacotes utilizados
!pip install requests_oauthlib
!pip install twython
!pip install nltk

In [ ]:
#modulos utilizado
from pyspark.streaming import StreamingContext
from pyspark import SparkContext
from requests_oauthlib import OAuth1Session
from operator import add
import requests_oauthlib
from time import gmtime, strftime
import requests
import time
import string
import ast
import json

In [ ]:
#pacotes nltk
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

In [ ]:
#Frequencia de update
INTERVALO_BATCH = 5

In [ ]:
#criando o StreamingContext
ssc = StreamingContext(sc, INTERVALO_BATCH)

In [ ]:
#lendo o arquivos texto e criando um RDD em memoria com Spark
arquivo = sc.textFile("D:\sparkinho-estudos\dsa\est01\dataset_analise_sentimento.csv")

In [ ]:
#removendo cabecalho
header = arquivo.take(1)[0]
dataset = arquivo.filter(lambda line: line != header)

In [ ]:
type(dataset)

In [ ]:
#Essa função separa as colunas em cada linha, criar uma tupla e remove a pontuacao
def get_row(line):
    row = line.split(',')
    sentimento = row[1]
    tweet = row[3].strip()
    translator = str.maketrans({key: None for key in string.punctuation})
    tweet = tweet.translate(translator)
    tweet = tweet.split(' ')
    tweet_lower = []
    for word in tweet:
        tweet_lower.append(word.lower())
    return (tweet_lower, sentimento)    

In [ ]:
#aplicar a funcao em cada linha do dataset
dataset_treino = dataset.map(lambda line: get_row(line)) 

In [ ]:
#criar um objeto SentimentAnalyzer
sentiment_analyzer = SentimentAnalyzer()

In [ ]:
nltk.download("stopwords")

In [ ]:
#Obtem a lista de stopwords em ingles
stopwords_all = []
for word in stopwords.words('english'):
    stopwords_all.append(word)
    stopwords_all.append(word + '_NEG')

In [ ]:
#obtem 10.000 tweets do dataset do treino e retorna todas as palavras que nao sao stopwords
dataset_treino_amostra = dataset_treino.take(10000)

In [ ]:
all_words_neg = sentiment_analyzer.all_words([mark_negation(doc) for doc in dataset_treino_amostra])
all_words_neg_nostops = [x for x in all_words_neg if x not in stopwords_all]

In [ ]:
#criar um unigram e extrai as features
unigram_feats = sentiment_analyzer.unigram_word_feats(all_words_neg_nostops, top_n = 200)
sentiment_analyzer.add_feat_extractor(extract_unigram_feats, unigrams = unigram_feats)
training_set = sentiment_analyzer.apply_features(dataset_treino_amostra)

In [ ]:
type(training_set)

In [ ]:
#treinar o modelo
trainer = NaiveBayesClassifier.train
classifier = sentiment_analyzer.train(trainer, training_set)

In [ ]:
#testa o classificar em algumas sentencas
test_sentence1 = [(['this', 'progam', 'is', 'bad'], '')]
test_sentence2 = [(['trough', 'day', 'at', 'work', 'today'], '')]
test_sentence3 = [(['good', 'wonderful', 'amazing', 'awesome'], '')]
test_set = sentiment_analyzer.apply_features(test_sentence1)
test_set2 = sentiment_analyzer.apply_features(test_sentence2)
test_set3 = sentiment_analyzer.apply_features(test_sentence3)

In [ ]:
#autenticacao do twitter
consumer_key = "yhJM02AXVkeJKbZ2UY9MZx2Pk"
consumer_secret = "bLNfYOuu0HJsca70Em3TQxoOAdudra0jsZI8kf1Rb7L74AxAjZ"
access_token = "164492542-xxFrS1MMJJreI8be6Y6rYT0ag0S5REGXfh52I8Ne"
access_token_secret = "kgYVf2UaqBRfQbn8Uaf7X2somaXAW8YglH9CpEkNnUTLV"

In [ ]:
#especifica a URL termo de busca
search_term = 'coronavirus'
sample_url = 'https://stream.twitter.com/1.1/statuses/sample.json'
filter_url = 'https://stream.twitter.com/1.1/statuses/filter.json?track='+search_term

In [ ]:
#criando um obejto de autenticacao para o twitter
auth = requests_oauthlib.OAuth1(consumer_key, consumer_secret, access_token, access_token_secret)

In [ ]:
#configurando o Stream
rdd = ssc.sparkContext.parallelize([0])
stream = ssc.queueStream([], default = rdd)

In [ ]:
type(stream)

In [ ]:
#total de tweets por update
NUM_TWEETS = 500

In [ ]:
#essa funcao conecta ao twitter e retorna um numero especifico de tweets (NUM_TWEETS)
def tfunc(t, rdd):
    return rdd.flatMap(lambda x: stream_twitter_data())

def stream_twitter_data():
    response = requests.get(filter_url, auth = auth, stream = True)
    print(filter_url, response)
    count = 0
    for line in response.iter-lines():
        try:
            if count > NUM_TWEETS:
                break
            post = json.loads(line.decode('utf-8'))
            contents = [post['text']]
            count += 1
            yield str(contents)
        except:   
            result = False

In [ ]:
stream = stream.transform(tfunc)

In [ ]:
coord_stream = stream.map(lambda line: ast.literal_eval(line))

In [ ]:
#essa funcao classifica os tweets, aplicando as features do modelo criado anteriormente
def classifica_tweet(tweet):
    sentence = [(tweet, '')]
    test_set = sentiment_analyzer.apply_features(sentence)
    print(tweet, classifier.classify(test_set[0][0]))
    return(tweet, classifier.classify(test_set[0][0]))

In [ ]:
#essa funcao retorna o texto do twitter
def get_tweet_text(rdd):
    for line in rdd:
        tweet = line.strip()
        translator = str.maketrans({key: None for key in string.punctuation})
        tweet = tweet.translate(translator)
        tweet = tweet.split(' ')
        tweet_lower = []
        for word in tweet:
            tweet_lower.append(word.lower())
        return(classifica_tweet(tweet_lower))    

In [ ]:
#criar uma lista vazia para os resultados
resultados = []

In [ ]:
#essa funcao salva o resultado dos batches de tweets junto com o timestamp
def output_rdd(rdd):
    global resultados
    pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
    counts = pairs.reduceByKey(add)
    output = []
    for count in counts.collect():
        output.append(count)
    result = [time.strftime("%I:%M:%S"), output]
    resultados.append(result)
    print(result)

In [ ]:
#a funcao foreachRDD() aplica uma funcao a cada RDD to streaming de dados
coord_stream.foreachRDD(lambda t, rdd: output_rdd(rdd))

In [ ]:
#strat streaming
ssc.start()
#ssc.awaitTermination()

In [ ]:
cont = True
while cont:
    if len(resultados) > 5:
        cont = False

In [ ]:
#grava os resultados
rdd_save = 'D:\sparkinho-estudos\dsa\est01\r'+time.strftime("%I:%M:%S")
resultados_rdd = sc.parallelize(resultados)
resultados_rdd.saveAsTextFile(rdd_save)

In [ ]:
#visualizar resultados
resultados_rdd.collect()

In [ ]:
#finaliza o streaming
ssc.stop()